In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Optimizer

In [2]:
data= pd.read_csv(r"C:\arun\Arun karthik.K_202113927\Bank_Personal_Loan_Modelling.csv")
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
x=data.drop(['ID','CreditCard'],axis=1).values
y=data['CreditCard'].values

In [4]:
model = Sequential()
model.add(Dense(16, input_dim=12, activation='relu'))
model.add(Dense(8,input_dim=16, activation='relu'))
model.add(Dense(6,input_dim=20, activation='sigmoid'))

In [5]:
class CulturalAlgorithm(Optimizer):
    def __init__(self, learning_rate=0.01, population_size=50, n_generations=10, n_best=5, n_elite=5, selection_rate=0.2, mutation_rate=0.1, **kwargs):
        super(CulturalAlgorithm, self).__init__(**kwargs)
        self.learning_rate = learning_rate
        self.population_size = population_size
        self.n_generations = n_generations
        self.n_best = n_best
        self.n_elite = n_elite
        self.selection_rate = selection_rate
        self.mutation_rate = mutation_rate

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'learning_rate': self.learning_rate,
            'population_size': self.population_size,
            'n_generations': self.n_generations,
            'n_best': self.n_best,
            'n_elite': self.n_elite,
            'selection_rate': self.selection_rate,
            'mutation_rate': self.mutation_rate,
        })
        return config

    def minimize(self, func):
        # Initialize the population with random weights
        population = [self._generate_weights() for i in range(self.population_size)]

        for generation in range(self.n_generations):
            # Evaluate the fitness of each individual in the population
            fitness = [func(weights) for weights in population]

            # Select the best individuals for breeding
            best_individuals = np.argsort(fitness)[:self.n_best]

            # Select the elite individuals for the next generation
            elite_individuals = np.argsort(fitness)[:self.n_elite]

            # Breed the best individuals
            offspring = []
            for i in range(self.population_size - self.n_elite):
                parents = np.random.choice(best_individuals, size=2, replace=False)
                offspring_weights = self._breed(population[parents[0]], population[parents[1]])
                offspring.append(offspring_weights)

            # Mutate the offspring
            for i in range(len(offspring)):
                if np.random.rand() < self.mutation_rate:
                    offspring[i] = self._mutate(offspring[i])

            # Add the elite individuals to the next generation
            next_generation = [population[i] for i in elite_individuals]

            # Add the offspring to the next generation
            next_generation += offspring

            # Replace the current population with the next generation
            population = next_generation

        # Evaluate the fitness of the final population
        fitness = [func(weights) for weights in population]

        # Return the best individual
        best_individual = population[np.argmin(fitness)]

        return best_individual

    def _generate_weights(self):
        # Generate random weights
        return [np.random.rand(12,16),np.random.rand(10)]

    def _breed(self, parent1, parent2):
        # Breed two individuals by averaging their weights
        offspring_weights = []
        for i in range(len(parent1)):
            offspring_weights.append((parent1[i] + parent2[i]) / 2.0)
        return offspring_weights
    
    def _mutate(self, individual):
        mutated_individual = []
        for i in range(len(individual)):
            r=np.random.rand()
            if r< 0.5:
                mutated_individual.append(individual[i]+np.random.normal(scale=self.learning_rate))
            else:
                mutated_individual.append(individual[i])
                return mutated_individual
                                        



In [6]:
def fitness_function(weights):
    model.set_weights(weights)

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=CulturalAlgorithm(), metrics=['accuracy'])
    from sklearn.model_selection import train_test_split
    x_train1, x_test1, y_train1, y_test1 = train_test_split(x, y, test_size=0.30, random_state=42)
    # Train the model
    model.fit(x_train1, y_train1, epochs=150)

    # Evaluate the model
    loss, accuracy = model.evaluate(X, y, verbose=0)

    # Return the negative of the accuracy (since this is a minimization problem)
    return accuracy

In [7]:
optimizer = CulturalAlgorithm()
best_weights = optimizer.minimize(fitness_function)

loss, accuracy = model.evaluate(x_test1, y_test1)


print("Accuracy:" ,accuracy)

ValueError: You called `set_weights(weights)` on layer "sequential" with a weight list of length 2, but the layer was expecting 6 weights. Provided weights: [array([[0.53029758, 0.2467722 , 0.96366715, 0.188...